# Data Wrangling with Spark SQL Quiz

This quiz uses the same dataset and most of the same questions from the earlier "Quiz - Data Wrangling with Data Frames Jupyter Notebook." For this quiz, however, use Spark SQL instead of Spark Data Frames.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) create a view to use with your SQL queries
# 5) write code to answer the quiz questions 

In [2]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd

In [3]:
spark = SparkSession \
    .builder \
    .appName("Data wrangling with Spark SQL Quiz") \
    .getOrCreate()

In [4]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

In [5]:
user_log.createOrReplaceTempView("user_log_table")

# Question 1

Which page did user id ""(empty string) NOT visit?

In [8]:
# TODO: write your code to answer question 1
spark.sql('''
          SELECT DISTINCT page
          FROM user_log_table 
          WHERE page NOT IN 
          (
              SELECT DISTINCT page 
              FROM user_log_table 
              WHERE userId = ""
          )
          '''
          ).show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

Why might you prefer to use SQL over data frames? Why might you prefer data frames over SQL?

# Question 3

How many female users do we have in the data set?

In [14]:
# TODO: write your code to answer question 3
spark.sql('''
          SELECT COUNT(DISTINCT userId)
          FROM user_log_table 
          WHERE gender = "F"
          '''
          ).show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [11]:
# TODO: write your code to answer question 4
spark.sql('''
          SELECT DISTINCT artist, COUNT(*) as songplayed
          FROM user_log_table 
          WHERE page = "NextSong"
          GROUP BY artist
          ORDER BY songplayed DESC
          LIMIT 1
          '''
          ).show()

+--------+----------+
|  artist|songplayed|
+--------+----------+
|Coldplay|        83|
+--------+----------+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [13]:
# TODO: write your code to answer question 5
spark.sql('''
          SELECT AVG(cnt)
          FROM
          (
              SELECT userId, songcnt, count(*) AS cnt
              FROM
              (
                  SELECT userId, ts, page, ishome, 
                      SUM(ishome) OVER 
                      (
                          PARTITION BY userId ORDER BY ts ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                      ) AS songcnt
                  FROM
                  (
                      SELECT userId, ts, page, CASE
                          WHEN page = "Home" THEN 1
                          ELSE 0
                          END AS ishome
                      FROM user_log_table
                  ) t
              ) t1
              WHERE page = "NextSong"
              GROUP BY userId, songcnt
          ) t2
          '''
          ).show()

+------------------+
|          avg(cnt)|
+------------------+
|6.9558333333333335|
+------------------+

